<a href="https://colab.research.google.com/github/aukabir/Machine-Learning-Project-2/blob/main/MLproject_2_diabetes_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Environment setup

In [8]:
import pandas as pd
import numpy
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns

import warnings # current version of seaborn generates a bunch of warnings that we'll ignore
warnings.filterwarnings("ignore")

2. Load and understanding data

2.1 Loading the dataset from Google Drive

In [9]:
data = pd.read_csv('/content/drive/MyDrive/DataBank for ML study circle/diabetes_prediction_dataset.csv')
data.head()

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,Female,80.0,0,1,never,25.19,6.6,140,0
1,Female,54.0,0,0,No Info,27.32,6.6,80,0
2,Male,28.0,0,0,never,27.32,5.7,158,0
3,Female,36.0,0,0,current,23.45,5.0,155,0
4,Male,76.0,1,1,current,20.14,4.8,155,0


2.2 Understanding the dataset

In [39]:
print('#Preview data:')
display(data.sample(3))
print('')
print('')
print('#Data Shape:')
display(data.shape)
print('')
print('')
print('#Data shapes, null values & data types:')
data.info()
print('')
print('')
print('#Five number summary of Data:')
display(data.describe().T)
print('')
print('')
print('#Diabetic prevalence:')
display(data.groupby('diabetes').size())

#Preview data:


,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
19489,Female,47.0,0,0,never,27.32,5.8,85,0
13447,Female,40.0,0,0,No Info,23.24,4.5,126,0
5804,Female,5.0,0,0,No Info,15.30,5.0,85,0




#Data Shape:


(100000, 9)



#Data shapes, null values & data types:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 9 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   gender               100000 non-null  object 
 1   age                  100000 non-null  float64
 2   hypertension         100000 non-null  int64  
 3   heart_disease        100000 non-null  int64  
 4   smoking_history      100000 non-null  object 
 5   bmi                  100000 non-null  float64
 6   HbA1c_level          100000 non-null  float64
 7   blood_glucose_level  100000 non-null  int64  
 8   diabetes             100000 non-null  int64  
dtypes: float64(3), int64(4), object(2)
memory usage: 6.9+ MB


#Five number summary of Data:


,count,mean,std,min,25%,50%,75%,max
age,100000.0,41.885856,22.516840,0.08,24.00,43.00,60.00,80.00
hypertension,100000.0,0.074850,0.263150,0.00,0.00,0.00,0.00,1.00
heart_disease,100000.0,0.039420,0.194593,0.00,0.00,0.00,0.00,1.00
bmi,100000.0,27.320767,6.636783,10.01,23.63,27.32,29.58,95.69
HbA1c_level,100000.0,5.527507,1.070672,3.50,4.80,5.80,6.20,9.00
blood_glucose_level,100000.0,138.058060,40.708136,80.00,100.00,140.00,159.00,300.00
diabetes,100000.0,0.085000,0.278883,0.00,0.00,0.00,0.00,1.00




#Diabetic prevalence:


diabetes
0    91500
1     8500
dtype: int64